### CS4423 - Networks
Angela Carnevale  
School of Mathematical and Statistical Sciences  
University of Galway

#### 1. Graphs and Graph Theory

# Week 3, lecture 1: 

# Bipartite Graphs, Block Matrices and Projections. 

In [ ]:
import networkx as nx
import numpy as np

In [ ]:
opts = { "with_labels": True, "node_color":'y' }

## Bipartite Graphs and colorings

A **(vertex)-coloring** of a graph $G$ is an assignment of (finitely many) colors to the nodes of $G$,
so that any two nodes which are connected by an edge have *different* colors.

* A graph is called **$N$-colorable**, if it has a vertex coloring with (at most) $N$ colors.

* The **chromatic number of a graph $G$** is smallest $N$ for which a graph $G$ is $N$-colorable.

**Theorem.** Let $G$ be a graph.  The following are equivalent:

* $G$ is bipartite;

* $G$ is $2$-colorable;
 
* each cycle in $G$ has even length.

(We'll give precise definitions of **cycle** and **length** in a bit)


The method `nx.bipartite.color` determines a $2$-coloring of a graph $G$ algorithmically, if it exists, i.e. if
$G$ is bipartite. 

For instance, consider a 2-dimensional $4\times 4$ grid:

In [ ]:
G44 = nx.grid_2d_graph(4, 4)
nx.draw(G44)

In [ ]:
color = nx.bipartite.color(G44)
color

In [ ]:
color = [color[x] for x in G44.nodes()]
color

In [ ]:
opts2 = { "with_labels": True, "node_color":color, "font_color":'r' }

In [ ]:
nx.draw(G44, **opts2)

**Note.** This won't work on a graph that is not $2$-colorable:

In [ ]:
nx.bipartite.color(nx.complete_graph(3))

## Affiliation Networks and Projections

Bipartite graphs arise in practice as models for **affiliation networks**.
In such a network, on one side of the graph we find people or *actors*, and on the other side attributes 
of the people, such as common interests (books bought online, TV shows watched), workplaces, social events attended ...
Edges in such network connect people with their attributes.



We construct a bipartite graph on the vertex set consisting of some of the first respondents to the survey and the 12 TV shows by using the following adjacency list:

In [ ]:
!cat data/tv_names.adj

In [ ]:
G=nx.read_adjlist("data/tv_names.adj")

In [ ]:
nx.draw(G, **opts)

In [ ]:
opts2 = { "with_labels": True, "font_color":'r' }

In [ ]:
color = nx.bipartite.color(G)
print(color)

In [ ]:
color = [color[i] for i in G.nodes()]
opts2["node_color"] = color

In [ ]:
actors = ['Angela','Evan','Andrew','Erika','M','John','Aoife','Jordan','Maciej','Sinead','Brian','Paulina','Nupur','Rana']
len(actors)

In [ ]:
foci = ['BB','BCS','DG','Succession','HIMYM','MrB','TO','Friends','XF','Lost','TB','GoT']
len(foci)

In [ ]:
nx.draw(G,nx.bipartite_layout(G,actors,align='vertical'),**opts2)

**Note.** The adjacency matrix $A$ of a bipartite graph $G$, with respect to a suitable ordering of the vertices
($X_1$ first, then $X_2$), has the form of a $2 \times 2$-block matrix,
$$
  A = \left( \begin{array}{cc} 0 & C \\ C^T & 0 \end{array} \right)
$$
where the blocks on the diagonal consist entirely of zeros, as there are no edges between vertices of the same color, and the lower left block is the **transpose** $C^T$ of the matrix $C$ of entries in the upper right. 

In [ ]:
M=nx.adjacency_matrix(G)
print(M.toarray())

To see the adjacency matrix in block form we need to give the nodes in a suitable order...

In [ ]:
H=nx.Graph()
H.add_nodes_from(actors)
H.add_nodes_from(foci)
H.nodes()

... and then import the edges from $G$.

In [ ]:
H.add_edges_from(G.edges())
print(list(H.edges()))

In [ ]:
AA=nx.adjacency_matrix(H)

In [ ]:
with np.printoptions(threshold=9999):
    print(AA.toarray())

* In `NetworkX`, all parts of a graph can have **attributes**: the nodes, 
the edges, and the graph object itself.  Graph object attributes of a graph `G` are stored in the field `G.graph`.  By convention, the two
underlying sets of a bipartite graph are stored here as attributes
called `'top'` and `'bottom'`.

* Here, we will simply construct lists of vertices from each of the two sets $X$ and $Y$ and construct a *biadjacency matrix* (this is all it's needed to (re)construct a bipartite graph!).

In [ ]:
X, Y = actors, foci
C = nx.bipartite.biadjacency_matrix(H, X, Y)
print(C.toarray())

As $A = A^T$, we get
\\[
A^T \cdot A = A \cdot A^T = A \cdot A = 
\left(
\begin{array}{cc}
C \cdot C^T & 0 \\ 0 & C^T \cdot C
\end{array}
\right)
\\]
where 
* $C \cdot C^T$ is (almost!) the adjacency matrix of the **projection** onto the vertex set $X$,
and 

* $C^T \cdot C$is (almost!) the adjacency matrix of the **projection** onto the vertex set $Y$.

In [ ]:
BB = nx.from_numpy_array((C*C.transpose()).toarray())
nx.draw(BB, **opts)

The loops are an indication that we've constructed a graph from a matrix that is not exactly an adjacency matrix:

In [ ]:
print(C*C.transpose().toarray())

We could use the usual method (fill the diagonal with $0$s and replace any positive number with a $1$) to get an actual adjacency matrix, or...

The `networkx` function `projected_graph` (taking input a bipartite graph and one of the two sets of vertices) does this for us:

In [ ]:
XX = nx.projected_graph(H, X)
nx.draw(XX, **opts)

In [ ]:
YY = nx.projected_graph(H,Y)
nx.draw(YY, **opts)

### All responses

And here's an implementation of the full bipartite graph from the results of the survey, and its projections on the set of TV shows under consideration.

In [ ]:
full_actors = [str(i) for i in range(1,21)]

In [ ]:
fullG = nx.read_adjlist('data/tv.adj')

In [ ]:
color = nx.bipartite.color(fullG)
print(color)

In [ ]:
color = [color[i] for i in fullG.nodes()]
opts2["node_color"] = color

In [ ]:
nx.draw(fullG,nx.bipartite_layout(fullG,foci,align='vertical'),**opts2)

In [ ]:
fullH=nx.Graph()
fullH.add_nodes_from(full_actors)
fullH.add_nodes_from(foci)
fullH.add_edges_from(fullG.edges())

In [ ]:
fullC = nx.bipartite.biadjacency_matrix(fullH, full_actors,foci)

In [ ]:
fullXX = nx.projected_graph(fullH, full_actors)
nx.draw(fullXX, **opts)

In [ ]:
fullYY = nx.projected_graph(fullH, foci)
nx.draw(fullYY, **opts)

Adjacency in the above graphs could be considered an indication of compatibility (between people, because they have watched shows in common; between shows, because adjacent ones might be appealing for like-minded people). Similar data structures are used to inform the typical "You might also like" boxes that we find in apps of streaming services or online shopping websites.

##  Code Corner

### `numpy`

* `array`: [[doc]](https://numpy.org/doc/stable/reference/generated/numpy.array.html)

* `transpose`: [[doc]](https://numpy.org/doc/stable/reference/generated/numpy.transpose.html)

* `printoptions`: [[doc]](https://numpy.org/doc/stable/reference/generated/numpy.printoptions.html) set options for printing arrays

### `networkx`

* `bipartite.color`: [[doc]](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.bipartite.basic.color.html) computes a $2$-coloring of a graph

* `bipartite_layout`: [doc](https://networkx.org/documentation/stable/reference/generated/networkx.drawing.layout.bipartite_layout.html) works out a useful way to draw a bipartite graph

* `bipartite.biadjacency_matrix`: [[doc]](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.bipartite.matrix.biadjacency_matrix.html) the incidence matrix of a bipartite graph

* `projected_graph`: [[doc]](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.bipartite.projection.projected_graph.html) the projected graph

